In [1]:
from pathlib import Path

WORKDIR = Path.cwd().absolute()
INPUTFILE = WORKDIR.parent / "day-15.txt"

with INPUTFILE.open("r") as file:
    inputs = [line.strip() for line in file.readlines()]

_inputs = [
]

for i in range(8):
    print(inputs[i])

from collections import namedtuple

Sensor = namedtuple("Sensor", ["x", "y", "r"])
sensors = []

for line in inputs:
    parts = line.split(" ")
    sx = int(parts[2][2:-1])
    sy = int(parts[3][2:-1])
    bx = int(parts[8][2:-1])
    by = int(parts[9][2:])
    sensors.append(Sensor(sx, sy, abs(sx-bx)+abs(sy-by)))

for i in range(8):
    print(sensors[i])


Sensor at x=3844106, y=3888618: closest beacon is at x=3225436, y=4052707
Sensor at x=1380352, y=1857923: closest beacon is at x=10411, y=2000000
Sensor at x=272, y=1998931: closest beacon is at x=10411, y=2000000
Sensor at x=2119959, y=184595: closest beacon is at x=2039500, y=-250317
Sensor at x=1675775, y=2817868: closest beacon is at x=2307516, y=3313037
Sensor at x=2628344, y=2174105: closest beacon is at x=3166783, y=2549046
Sensor at x=2919046, y=3736158: closest beacon is at x=3145593, y=4120490
Sensor at x=16, y=2009884: closest beacon is at x=10411, y=2000000
Sensor(x=3844106, y=3888618, r=782759)
Sensor(x=1380352, y=1857923, r=1512018)
Sensor(x=272, y=1998931, r=11208)
Sensor(x=2119959, y=184595, r=515371)
Sensor(x=1675775, y=2817868, r=1126910)
Sensor(x=2628344, y=2174105, r=913380)
Sensor(x=2919046, y=3736158, r=610879)
Sensor(x=16, y=2009884, r=20279)


In [8]:
from typing import List

def mdist(x1, y1, x2, y2):

    return abs(x1-x2) + abs(y1-y2)

class Rect:

    def __init__(self, left:int, width:int, bottom:int, height:int) -> None:

        self.left = left
        self.right = left + width
        self.bottom = bottom
        self.top = bottom + height

        return

    @property
    def width(self):

        return self.right - self.left

    @property
    def height(self):

        return self.top - self.bottom

    @property
    def area(self):

        return self.width * self.height

    def inrange(self, sensor:Sensor) -> bool:

        return (mdist(self.left,    self.bottom, sensor.x, sensor.y) <= sensor.r) and \
               (mdist(self.right-1, self.bottom, sensor.x, sensor.y) <= sensor.r) and \
               (mdist(self.right-1, self.top-1,  sensor.x, sensor.y) <= sensor.r) and \
               (mdist(self.left,    self.top-1,  sensor.x, sensor.y) <= sensor.r)

    def split(self):

        mx = self.left   + (self.width  // 2)
        my = self.bottom + (self.height // 2)

        return [
            Rect(self.left, mx-self.left,  self.bottom, my - self.bottom),  # lower left
            Rect(self.left, mx-self.left,  my,          self.top - my),     # upper left
            Rect(mx,        self.right-mx, my,          self.top - my),     # upper right
            Rect(mx,        self.right-mx, self.bottom, my - self.bottom)   # lower right
        ]

    def __str__(self):

        return f"({self.left},{self.bottom})-({self.right},{self.top}) : {self.width}×{self.height} = {self.area}"

    def uncovered(self, sensors:List[Sensor]) -> List["Rect"]:

        # print(self)

        if self.area == 0:
            return []

        if any(self.inrange(sensor) for sensor in sensors):
            return []

        if self.area == 1:
            return [self]

        results = []
        for subrect in self.split():
            results += subrect.uncovered(sensors)

        return results

for rect in Rect(0, 4_000_001, 0, 4_000_000).uncovered(sensors):
    print(rect)


(3157535,3363767)-(3157536,3363768) : 1×1 = 1


In [22]:
r = Rect(10410,2,1999999,2)
print("\n".join(map(str,r.split())))
print("\n".join(map(str,Rect(10410,1,1999999,1).split())))
print("\n".join(map(str,Rect(0,4000001,0,4000001).split())))

(10410,1999999)-(10411,2000000) : 1×1 = 1
(10410,2000000)-(10411,2000001) : 1×1 = 1
(10411,2000000)-(10412,2000001) : 1×1 = 1
(10411,1999999)-(10412,2000000) : 1×1 = 1
(10410,1999999)-(10410,1999999) : 0×0 = 0
(10410,1999999)-(10410,2000000) : 0×1 = 0
(10410,1999999)-(10411,2000000) : 1×1 = 1
(10410,1999999)-(10411,1999999) : 1×0 = 0
(0,0)-(2000000,2000000) : 2000000×2000000 = 4000000000000
(0,2000000)-(2000000,4000001) : 2000000×2000001 = 4000002000000
(2000000,2000000)-(4000001,4000001) : 2000001×2000001 = 4000004000001
(2000000,0)-(4000001,2000000) : 2000001×2000000 = 4000002000000
